**Vanessa Navarro Coronado e Iván Sánchez Castellanos**

https://www.kaggle.com/kanncaa1/roc-curve-with-k-fold-cv

# Práctica 2 - Clasificación supervisada en scikit-learn --> VOICE DATASET

En esta práctica vamos a estudiar el dataset Voice. Probaremos los clasificadores de Árbol de Decisión y KNN, y analizaremos cuál es mejor en cada caso. Además, trataremos de descubrir las configuraciones óptimas de dichos algoritmos, y realizaremos un breve estudio sobre ellos.

Además, en la parte extra utilizaremos un Naive Bayes, usando el mismo procedimiento que seguimos para los clasificadores anteriormente citados.

Para comenzar, como siempre, lo primero que hacemos es importar los paquetes necesarios, e inicializar nuestra semilla para nuestros experimentos.

In [3]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [4]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [5]:
seed=6342
np.random.seed(6342)

### Cargado de los datos

A continuación, cargamos los datos del dataset Voice. Este será separado en dos conjuntos, uno con los atributos y otro con la clase del dataset. Esto es necesario para que la libreria scikit learn pueda ser utilizada.

In [6]:
# Update the file path to fit your system
dfVoice = pd.read_csv("../data/voice.csv", dtype={ "label": 'category'})
dfAttributesVoice = dfVoice.drop('label', 1)
dfLabelVoice = dfVoice['label']

Una vez cargados los datasets, debemos dividirlos en dos conjuntos de Train y Test para poder realizar nuestros experimentos.

In [7]:
# Divide into train/test split for our experiments
from sklearn.model_selection import train_test_split
train_attsVoice, test_attsVoice, train_labelVoice, test_labelVoice = train_test_split( 
    dfAttributesVoice,
    dfLabelVoice,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelVoice)

# 1. Selección y evaluación de modelos

En esta sección realizaremos experimentos utilizando el algoritmo GridSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN para el dataset Voice.

In [8]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

Los pasos a seguir en este apartado son:
* Crear un objeto de la clase GridSearchCV, al que le pasaremos:
    - el estimador que vamos a utilizar (DecisionTree o KNN), 
    - los hiperparámetros del clasificador que queremos tener en cuenta a la hora de buscar la configuración óptima.
    - También debemos indicar qué tipo de métrica usaremos para valorar las configuraciones,
    - cuántos folds se deben realizar en el proceso de validación cruzada que realiza el GridSearchCV. Si en este campo especificamos un objeto de la clase StratifiedKFold con 10 folds y nuestra semilla, podrá realizar un proceso de validación cruzada estratificada.
    - iid = False, para que la evaluación de los resultados se haga sobre una media aritmética, y no sobre una media ponderada.
* Entrenar y validar el clasificador con los datos de nuestro dataset, y analizar los resultados obtenidos (accuracy, precision y recall).

## 1.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

Comenzamos con el clasificador DecisionTree.

In [9]:
# Cargamos el arbol de decision
from sklearn import tree

from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

### 1.1.1. GridSearch

Aquí definimos nuestro GridSearch con los parámetros elegidos. Como estamos usando el clasificador DecisionTree, podemos pasarle como parámetro de 'random_state' nuestra semilla para poder hacer reproducibles nuestros experimentos. 

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [10]:
clfTree = GridSearchCV(
    estimator = tree.DecisionTreeClassifier(random_state=seed),
    param_grid =
        {'criterion': ["entropy","gini"],'max_depth': [3, 5, 10, None], 'min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

Si asignamos el valor 'None' al parámetro 'max_depth', el clasificador intentará llegar a la máxima profundidad del árbol por defecto.

Lo ejecutamos con el dataset para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro GridSearch.

In [11]:
# We can fit and use the pipeline as usual
clsTreeVoice = clfTree.fit(train_attsVoice, train_labelVoice)
predictionTreeVoice = clfTree.predict(test_attsVoice)
print('Accuracy Voice Tree:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoice)

Accuracy Voice Tree:


0.9605678233438486

In [12]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(clfTree, train_attsVoice, train_labelVoice).mean()

0.96488353332661081

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el GridSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 5, y un número mínimo de ejemplos por hoja igual a 10.

In [13]:
print('Mejor configuración de parámetros (Voice Tree):')
clsTreeVoice.best_params_

Mejor configuración de parámetros (Voice Tree):


{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida.

In [14]:
print('Confusion matrix Voice Tree:')
metrics.confusion_matrix(test_labelVoice, predictionTreeVoice)

Confusion matrix Voice Tree:


array([[307,  10],
       [ 15, 302]])

## 1.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador.

In [15]:
from sklearn import neighbors

In [16]:
import sklearn.metrics as metrics

In [17]:
from sklearn.model_selection import GridSearchCV

### 1.2.1. GridSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el GridSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el GridSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [18]:
clfKNN = GridSearchCV(
    estimator = neighbors.KNeighborsClassifier(),
    param_grid = 
        { 'n_neighbors' : [1,2,3,4,5], 'weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset.

In [19]:
# We can fit and use the pipeline as usual
clsKNNVoice = clfKNN.fit(train_attsVoice, train_labelVoice)
predictionKNNVoice = clfKNN.predict(test_attsVoice)
print('Accuracy Voice KNN:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoice)

Accuracy Voice KNN:


0.71135646687697163

In [20]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(clfKNN, train_attsVoice, train_labelVoice).mean()

0.69771044110562119

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el GridSearch. En este caso es utilizar 4 vecinos, usando la métrica distance para medir las distancias.

In [21]:
print('Mejor configuración de parámetros (Voice KNN):')
clsKNNVoice.best_params_

Mejor configuración de parámetros (Voice KNN):


{'n_neighbors': 4, 'weights': 'distance'}

A continuación mostramos la matriz de confusión obtenida.

In [22]:
print('Confusion matrix Voice KNN:')
metrics.confusion_matrix(test_labelVoice, predictionKNNVoice)

Confusion matrix Voice KNN:


array([[215, 102],
       [ 81, 236]])

## 1.3. Comparativa

En este apartado intentaremos analizar cuál de los clasificadores ha obtenido mejores resultados en el dataset estudiado.

En el dataset Pima los resultados de accuracy para el árbol de decisión y el KNN con k=5, son: 
* Accuracy Voice Tree: 0.9605678233438486
* Accuracy Voice 4-NN: 0.71135646687697163

Por tanto, podemos concluir que el mejor modelo es el obtenido con el árbol de decisión.

## 1.4. Estudio de los algoritmos

En este apartado vamos a estudiar invidualmente los algoritmos:
* En KNN, estudiaremos los parámetros aprendidos por el clasificador, realizando varias pruebas con distintas configuraciones de parámetros y analizando los resultados. Analizaremos el dataset Voice, y compararemos con el estudio realizado con Pima y Wisconsin.
* Para el árbol, estudiaremos los parámetros aprendidos por el clasificador, y también su estructura. Analizaremos las estructuras del clasificador con la configuración óptima y otra con una configuración suboptima, y valoraremos los resultados.

Para analizarlos, primero vamos a crear una funcion getScores que será similar a nuestra implementación del GridSearch pero más sencilla. Simplemente obtendremos los scores usando los distintos parámetros del clasificador. No realizamos proceso de validación cruzada, solamente usamos el holdout que teníamos al principio de la práctica.

Después obtenemos tiempo y accuracy para las configuraciones, y realizaremos una comparativa de todos ellos.

In [23]:
from itertools import product
from time import time

def getScores(estim,paramG,train_atts,train_label,test_atts,test_label):
    
    scores=[] #para almacenar los resultados de accuracy a devolver
    exTimes=[] #para almacenar los tiempos a devolver
    
    #Generamos las configuraciones de parametros como en GridS
    items = paramG.items()
    keys, values = zip(*items)
    v = list(product(*values))
    
    # Recorremos las configuraciones y almacenamos los resultados
    for param in v:
        
        params = dict(zip(keys,param))
        estim.set_params(**params)
        
        timeSum=0 #variable intermedia para mostrar la media de los tiempos
        
        for i in range(0,100):
            start = time()
            estim.fit(train_atts,train_label) #entrenamos con el conjunto de Train
            predictions = estim.predict(test_atts) #array de predicciones para el conjunto de Test
            end = time()
            timeSum += (end - start)
        exTimes.append(timeSum/100) #añadimos al array de tiempos la media de las 100 medidas de tiempo realizadas
        
        comparison = np.sum(predictions == test_label) #comparamos predicciones con test_label y sumamos los aciertos
        accuracy = comparison / len(predictions) #dividimos entre los casos totales
        scores.append(accuracy) #añadimos al array de scores el accuracy obtenido con la configuración de parámetros actual
        
    return (v,scores,exTimes) #devolvemos las configuraciones, los accuracys y los tiempos de cada configuracion

### 1.4.1. Estudio del algoritmo KNN

Con este clasificador, usaremos las combinaciones teniendo en cuenta el número de vecinos (k=1,5,10,50,100) y el método de evaluación de las distancias (uniforme o inversa de la distancia). Para ello usaremos nuestra función getScores.

#### 1.4.1.1. KNN con Voice

Una vez hecho esto, podemos hacer una comparativa de los resultados obtenidos para todas las configuraciones del clasificador KNN.

In [24]:
paramsKNNVoice,scoresKNNVoice,exTimesKNNVoice=getScores(neighbors.KNeighborsClassifier(),
                                                     { 'weights': ['uniform','distance'], 'n_neighbors' : [1,5,10,50,100] },
                                                     train_attsVoice,train_labelVoice,
                                                     test_attsVoice,test_labelVoice)
resultadosKNNVoice=pd.DataFrame(list(zip(paramsKNNVoice,scoresKNNVoice,exTimesKNNVoice)))
resultadosKNNVoice.columns=['paramsKNNVoice','scoresKNNVoice','exTimesKNNVoice']
resultadosKNNVoice

,paramsKNNVoice,scoresKNNVoice,exTimesKNNVoice
0,"(uniform, 1)",0.695584,0.008447
1,"(uniform, 5)",0.708202,0.009291
2,"(uniform, 10)",0.703470,0.011899
3,"(uniform, 50)",0.679811,0.016622
4,"(uniform, 100)",0.692429,0.023539
5,"(distance, 1)",0.695584,0.008079
6,"(distance, 5)",0.709779,0.009332
7,"(distance, 10)",0.714511,0.010247
8,"(distance, 50)",0.690852,0.020599
9,"(distance, 100)",0.689274,0.034512


Como podemos comprobar, la configuración con mejor accuracy sería usar k=10 y la inversa de la distancia (fila 7 de la tabla). Por tanto, si aumentamos K, podríamos pensar que se mejora el accuracy, pero solo hasta cierto punto, porque con K=100 (fila 9) ya obtenemos un accuracy peor. De forma general, usar un K muy alto se asemeja a un clasificador ZeroR, ya que ambos utilizarían la estrategia de clasificación por la clase mayoritaria.

Además, debemos tener en mente que cuanto mayor sea el parámetro K, mayor será el tiempo de ejecución. En este caso, el tiempo de ejecución se corresponde con el tiempo de realizar la predicción con el conjunto de Test. Esto es debido a que el entrenamiento en el clasificador KNN es muy rápido, ya que solo consiste en copiar la base de datos de los casos de Train.

### 1.4.2. Estudio del algoritmo DecisionTree

Con este clasificador, realizaremos las combinaciones teniendo en cuenta el criterio (entropía o gini), la profundidad máxima del árbol, y el número mínimo de ejemplos por hoja. 

Para analizar la estructura de los árboles generados con las diferentes combinaciones de parámetros, vamos a usar (además de nuestra función getScores) la función getTreeNodes, que nos devuelve el número de nodos que presenta un árbol.

In [25]:
def getTreeNodes(estim,paramG,train_atts,train_label):
    
    nodes=[] #para almacenar el numero de nodos de cada arbol generado
    
    #Generamos las configuraciones de parametros como en GridS
    items = paramG.items()
    keys, values = zip(*items)
    v = list(product(*values))
    
    # Recorremos las configuraciones y almacenamos los resultados
    for param in v:
        
        params = dict(zip(keys,param))
        estim.set_params(**params)

        estim.fit(train_atts,train_label) #entrenamos con el conjunto de Train
        
        nodes.append(estim.tree_.node_count) #añadimos al array el numero de nodos del arbol actual
        
    return (v,nodes) #devolvemos las configuraciones de los arboles y el numero de nodos de cada uno de ellos

#### 1.4.2.1. Árbol con Voice

Vamos a hacer una comparativa de los resultados obtenidos para todas las configuraciones del árbol de clasificación, junto con la complejidad de cada árbol generado (representada por el número de nodos de cada árbol).

In [55]:
paramsTreeVoice,scoresTreeVoice,exTimesTreeVoice=getScores(tree.DecisionTreeClassifier(random_state=seed),
                                                        {'criterion': ["entropy","gini"],
                                                         'max_depth': [3, 5, 10], 
                                                         'min_samples_leaf': [3,5,10]},
                                                        train_attsVoice,train_labelVoice,
                                                        test_attsVoice,test_labelVoice)
resultadosTreeVoice=pd.DataFrame(list(zip(paramsTreeVoice,scoresTreeVoice,exTimesTreeVoice)))
resultadosTreeVoice.columns=['paramsTreeVoice','scoresTreeVoice','exTimesTreeVoice']

In [56]:
paramsTreeVoice,nodesTreeVoice=getTreeNodes(tree.DecisionTreeClassifier(random_state=seed),
                                          {'criterion': ["entropy","gini"],
                                           'max_depth': [3, 5, 10], 
                                           'min_samples_leaf': [3,5,10]},
                                          train_attsVoice,train_labelVoice)
resultadosTreeVoice.assign(nodesTreeVoice = nodesTreeVoice)

,paramsTreeVoice,scoresTreeVoice,exTimesTreeVoice,nodesTreeVoice
0,"(entropy, 3, 3)",0.944795,0.032279,15
1,"(entropy, 3, 5)",0.944795,0.030422,15
2,"(entropy, 3, 10)",0.944795,0.030794,15
3,"(entropy, 5, 3)",0.957413,0.039179,53
4,"(entropy, 5, 5)",0.957413,0.038390,51
5,"(entropy, 5, 10)",0.960568,0.037523,41
6,"(entropy, 10, 3)",0.955836,0.040192,91
7,"(entropy, 10, 5)",0.957413,0.039979,77
8,"(entropy, 10, 10)",0.962145,0.038568,53
9,"(gini, 3, 3)",0.962145,0.015112,15


Como vemos en la tabla, la configuración con mayor accuracy sería la que se encuentra en la fila 16: criterio gini, máxima profundidad del árbol igual a 10 y mínimo número de ejemplos por hoja igual a 5.

Sin embargo, esta configuración genera un árbol muy complejo, con más de 80 nodos. Por tanto, podríamos compararlo con el segundo mejor árbol generado (árbol de la fila 14 de la tabla), el cual presenta un accuracy algo más bajo, pero está compuesto por la mitad de nodos. Este caso necesitaría una profundidad máxima del árbol igual a 5, y un número mínimo de ejemplos por hoja igual a 10, para conseguir (casi) igualar el accuracy al del árbol de la fila 16, por lo que sería interesante seleccionar el árbol más simple para nuestra clasificación. Además, su tiempo de ejecución es menor.

De esta forma podemos ver que debemos tener en cuenta otros factores además del accuracy para poder evaluar la efectividad del clasificador.

### 1.4.3. Conclusiones del estudio de los algoritmos

Tras el estudio de los algoritmos, nos hemos dado cuenta de que dependiendo del problema y del dataset al que nos enfrentemos, es posible que la configuración de parámetros óptima para un determinado algoritmo no siempre sea la misma. 
* En el KNN con el dataset Pima hemos visto que es bueno aumentar la vecindad (hasta cierto punto). Sin embargo, con el dataset Wisconsin los resultados empeoran mucho al usar vecindades grandes, y además se consumía demasiado tiempo.
* Con el árbol de decisión, hemos aprendido que es una buena práctica observar las configuraciones subóptimas del clasificador para obtener árboles más sencillos, en los cuales el tiempo de ejecución es menor y obtienen rendimientos similares a árboles más grandes.
* Con este nuevo dataset Voice, hemos visto que usando KNN como clasificador, nos ocurre algo parecido a lo que pasaba con Pima, y en el árbol hemos llegado a la misma conclusión que llegamos con Pima y Wisconsin.

# 2. Implementación de GridSearch manualmente

En este apartado realizamos una implementación manual de un GridSearh básico, con su constructor, y las funciones fit y predict, que son las que utilizamos en esta práctica.

Nuestra clase GridS tiene como atributos:
* estim: estimador o clasificador a evaluar,
* crossval: cómo realizar el proceso de validación cruzada durante la evaluación,
* score: tipo de métrica a utilizar para la evaluación de la configuración óptima,
* keys: nombre de los parámetros del clasificador a tener en cuenta en la evaluación,
* v: lista con todas las posibles combinaciones de parámetros del clasificador,
* bestScore: mejor puntuación obtenida con la configuración de parámetros del clasificador,
* bestParams: mejor configuración de parámetros del clasificador.

La función fit se encarga de realizar la validación cruzada para cada una de las diferentes combinaciones de parámetros contenidas en self.v, obteniendo así aquella configuración que ofrezca una mayor puntuación. Una vez obtenida, entrena el clasificador con el conjunto de datos que se le pasa como parámetro a la función fit.

La función predict simplemente hace la validación del clasificador con el conjunto que reciba como parámetro.

In [28]:
from itertools import product
from sklearn.model_selection import cross_val_score,StratifiedKFold

class GridS(object):
    def __init__(self,estim, paramG, score, crossval):
        items = paramG.items()
        self.keys, values = zip(*items)
        self.v = list(product(*values))
        self.estim=estim
        self.crossval=crossval
        self.score=score
            
    def fit(self,atts,label):
        self.bestScore=0
        for param in self.v:
            params = dict(zip(self.keys,param))
            self.estim.set_params(**params)
            scores = cross_val_score(estimator=self.estim, 
                                     X=atts, y=label, 
                                     scoring=self.score, 
                                     cv=self.crossval)
            if self.bestScore < scores.mean():
                self.bestScore=scores.mean()
                self.bestParams=params
        self.estim.set_params(**self.bestParams)
        self.estim.fit(atts,label)
    
    def predict(self, atts):
        return self.estim.predict(atts)

## 2.1. KNN con GridSearch manual

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos un clasificador KNN, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al número de vecinos (de 1 a 5 vecinos), y el tipo de métrica a utilizar para valorar la distancia (uniforme o inversa de la distancia). El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla. 

In [29]:
gsKNN = GridS(
    estim = neighbors.KNeighborsClassifier(),
    paramG = 
        { 'n_neighbors' : [1,2,3,4,5], 'weights': ['uniform','distance'] },
    score = 'accuracy',
    crossval = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

A continuación utilizamos nuestro GridS en el dataset y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [30]:
clsKNNVoiceGS = gsKNN.fit(train_attsVoice, train_labelVoice)
predictionKNNVoiceGS = gsKNN.predict(test_attsVoice)
print('Accuracy Voice KNN + GridS:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoiceGS)

Accuracy Voice KNN + GridS:


0.71135646687697163

In [31]:
print('Mejor configuración de parámetros (Voice KNN + GridS):')
gsKNN.bestParams

Mejor configuración de parámetros (Voice KNN + GridS):


{'n_neighbors': 4, 'weights': 'distance'}

## 2.2. Árbol de decisión con GridSearch manual

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos un clasificador DecisionTree, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al criterio de evaluación (entrioía o gini), la máxima profundidad del árbol, y el número mínimo de ejemplos por hoja. El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla.

In [32]:
gsTree=GridS(
    estim = tree.DecisionTreeClassifier(random_state=seed),
    paramG={'criterion': ["entropy","gini"],'max_depth': [3, 5, 10, None], 'min_samples_leaf': [3,5,10]},
    score='accuracy',
    crossval=StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

A continuación utilizamos nuestro GridS en el dataset y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [33]:
# We can fit and use the pipeline as usual
clsTreeVoiceGS = gsTree.fit(train_attsVoice, train_labelVoice)
predictionTreeVoiceGS = gsTree.predict(test_attsVoice)
print('Accuracy Voice Tree + GridS:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoiceGS)

Accuracy Voice Tree + GridS:


0.9605678233438486

In [34]:
print('Mejor configuración de parámetros (Voice Tree + GridS):')
gsTree.bestParams

Mejor configuración de parámetros (Voice Tree + GridS):


{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}

# 4. Extras: Algoritmo RandomizedSearchCV

In [35]:
from sklearn.model_selection import RandomizedSearchCV

En esta sección realizaremos experimentos utilizando el algoritmo RandomizedSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN para nuestro dataset. Seguiremos el mismo proceso que hicimos con GridSearch.

## 4.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

### 4.1.1. RandomizedSearch

Aquí definimos nuestro RandomizedSearch con los parámetros elegidos. Como estamos usando el clasificador DecisionTree, podemos pasarle como parámetro de 'random_state' nuestra semilla para poder hacer reproducibles nuestros experimentos. 

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [36]:
clfTreeRS = RandomizedSearchCV(
    estimator = tree.DecisionTreeClassifier(random_state=seed),
    param_distributions =
        {'criterion': ["entropy","gini"],'max_depth': [3, 5, 10, None], 'min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

Ejecutamos nuestro gridsearch con el dataset para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro RandomizedSearch.

In [53]:
# We can fit and use the pipeline as usual
clsTreeVoiceRS = clfTreeRS.fit(train_attsVoice, train_labelVoice)
predictionTreeVoiceRS = clfTreeRS.predict(test_attsVoice)
print('Accuracy Voice Tree with RS:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoice)

Accuracy Voice Tree with RS:


0.9605678233438486

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el GridSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 5, y un número mínimo de ejemplos por hoja igual a 5.

In [54]:
print('Mejor configuración de parámetros (Voice Tree) con RS:')
clfTreeRS.best_params_

Mejor configuración de parámetros (Voice Tree) con RS:


{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 5}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida.

In [39]:
print('Confusion matrix Voice Tree with RS:')
metrics.confusion_matrix(test_labelVoice, predictionTreeVoiceRS)

Confusion matrix Voice Tree with RS:


array([[305,  12],
       [ 15, 302]])

## 4.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador.

### 4.2.1. RandomizedSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el RandomizedSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el RandomizedSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [40]:
clfKNNRS = RandomizedSearchCV(
    estimator = neighbors.KNeighborsClassifier(),
    param_distributions = 
        { 'n_neighbors' : [1,2,3,4,5], 'weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset.

In [51]:
# We can fit and use the pipeline as usual
clsKNNVoiceRS = clfKNNRS.fit(train_attsVoice, train_labelVoice)
predictionKNNVoiceRS = clfKNNRS.predict(test_attsVoice)
print('Accuracy Voice KNN with RS:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoiceRS)

Accuracy Voice KNN with RS:


0.71135646687697163

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el RandomizedSearch. En este caso es utilizar 4 vecinos, usando la métrica distance para medir las distancias.

In [52]:
print('Mejor configuración de parámetros (Voice KNN) con RS:')
clfKNNRS.best_params_

Mejor configuración de parámetros (Voice KNN) con RS:


{'n_neighbors': 4, 'weights': 'distance'}

A continuación mostramos la matriz de confusión obtenida.

In [43]:
print('Confusion matrix Voice KNN with RS:')
metrics.confusion_matrix(test_labelVoice, predictionKNNVoiceRS)

Confusion matrix Voice KNN with RS:


array([[215, 102],
       [ 81, 236]])

## 4.3. Naive Bayes con GridSearch

Ahora vamos a usar el GridSearchCV con el clasificador Naive Bayes, concretamente el algoritmo GaussianNB, el cual se utiliza para clasificar con variables continuas. Este algoritmo solo recibe como parámetro 'priors' las probabilidades a priori de la clase. En nuestro caso, crearemos un GridSearch con el parámetro 'priors' fijado a diferentes valores.

In [44]:
from sklearn.naive_bayes import GaussianNB

In [45]:
clfNB = GridSearchCV(
    estimator = GaussianNB(),
    param_grid = 
        { 'priors':[[0.3,0.7],[0.5,0.5],[0.9,0.1],[0.6,0.4]] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False
)

Primero entrenamos nuestro clasificador con el dataset, y observamos que el accuracy es menor que el obtenido con el árbol de decisión, y peor que el obtenido con el KNN.

In [48]:
# We can fit and use the pipeline as usual
clsNBVoice = clfNB.fit(train_attsVoice, train_labelVoice)
predictionNBVoice = clfNB.predict(test_attsVoice)
print('Accuracy Voice NB:')
metrics.accuracy_score(test_labelVoice, predictionNBVoice)

Accuracy Voice NB:


0.8848580441640379

Imprimimos la mejor configuración de parámetros obtenida por el GridSearch.

In [49]:
print('Mejor configuración de parámetros (Voice NB):')
clsNBVoice.best_params_

Mejor configuración de parámetros (Voice NB):


{'priors': [0.3, 0.7]}